In [11]:
import data
import model
import numpy as np
from random import choice, seed
from model import build_model, dense, dropout, prelu

In [2]:
np.random.seed(1337)  # TODO repeatability
seed(1337)

In [3]:
X, y, _, _, _ = data.load("~/Documents/flavours-of-physics-start/input/training.csv",
                          shuffle=True)
Xa, ya, wa, _, _ = data.load("~/Documents/flavours-of-physics-start/input/check_agreement.csv",
                             shuffle=False, weight=True)
Xc, _, _, mc, _ = data.load("~/Documents/flavours-of-physics-start/input/check_correlation.csv",
                            shuffle=False, mass=True, test=True)

X, scaler = data.preprocess_data(X)
Xa, _ = data.preprocess_data(Xa)
Xc, _ = data.preprocess_data(Xc)

X_test, _, _, _, ids = data.load("~/Documents/flavours-of-physics-start/input/test.csv", test=True,
                                 ids=True)
X_test = scaler.transform(X_test)
print "Done"

Done


In [30]:
def create_model(input_size, l=0.5, domain_loss_weight=0.2):

    model = build_model(input_size,
                        [dense(100), dropout(.32),
                         dense(200), prelu(), dropout(.27),
                         dense(150), prelu(), dropout(.22),
                         dense(75), prelu(), dropout(.17),
                         dense(2)],
                        [dense(150), prelu(), dropout(0.10),
                         dense(100), prelu(), dropout(0.05),
                         dense(2)], 0, l, domain_loss_weight=domain_loss_weight)
    return model

In [ ]:
n_models = 1
n_epochs = 50
probs = None
for i in range(n_models):
    l = 0.2
    domain_weight = 5.2
    print "Model {}; l = {}".format(i, l)
    m = create_model(X.shape[1], l, domain_weight)
    model.fit_model(m, X, y, Xa, ya, wa, Xc, mc, epoch_count=n_epochs, batch_size=64,
                    validation_split=0.02, verbose=2)
    p = model.predict_model(m, np.array(X_test))
    probs = p if probs is None else p + probs
probs /= n_models

Model 0; l = 0.2
Train on 66201 samples, validate on 1352 samples
Epoch 1/50
KS: 0.095434500702 : 0.09 / CvM: 0.00107163475567 : 0.002
13s - loss: 2.0888 - val_loss: 1.9806
Epoch 2/50
KS: 0.0918419097765 : 0.09 / CvM: 0.00110387695275 : 0.002

In [ ]:
# random_classifier = np.random.rand(len(probs))
# q = 0.98
# combined_probs = q * (probs ** 30) + (1 - q) * random_classifier

In [29]:
data.save_submission(ids, probs, "grl_prediction.csv")